# 1- Instale e Importe os Pacotes / Install and Import Packages

In [ ]:
!pip install scipy tqdm

In [ ]:
import os
import scipy.io # Adicionado scipy.io para carregar arquivos .mat
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader # Importar Dataset
from torchvision import transforms, models
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from PIL import Image # Importar Imagem

# 2- Baixando, Extraindo e Organizando


Dataset Oxford Flowers 102

In [ ]:
# Download and extract the dataset
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat

!mkdir /content/Dataset_Flowers
!tar -xzf 102flowers.tgz -C /content/Dataset_Flowers
!mv imagelabels.mat /content/Dataset_Flowers/
!mv setid.mat /content/Dataset_Flowers/

print("Dataset downloaded and extracted to /content/Dataset_Flowers")

--2025-08-25 21:19:07--  https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/flowers/102/102flowers.tgz [following]
--2025-08-25 21:19:07--  https://thor.robots.ox.ac.uk/flowers/102/102flowers.tgz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/octet-stream]
Saving to: ‘102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  10.0MB/s    in 27s     

2025-08-25 21:19:35 (12.1 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]

--2025-08-25 21:19:35--  https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels

# 3 - Carrega e Extraia os Arquivos do Dataset / Upload and Extract Dataset Files

In [4]:
#Extraia as imagens
#Extract the images

data_dir = "/content/Dataset_Flowers"
img_dir = os.path.join(data_dir, "jpg") # Diretório contendo arquivos de imagem

# 4- Carrega os Rótulos e as Divisões do Dataset / Load Labels and Dataset Splits

Os rótulos e as divisões do Dataset são armazenadas em arquivos .mat (formato MatLab). Você carrega eles usando scipy.io.        

The labels and data splits are stored in .mat files (MatLab format). You load them using scipy.io.

In [5]:
#Carrega arquivos .mat
#Load .mat files

labels = scipy.io.loadmat(os.path.join(data_dir, "imagelabels.mat"))['labels'][0] - 1
split = scipy.io.loadmat(os.path.join(data_dir, "setid.mat"))

train_ids = split['trnid'][0] - 1
val_ids = split['valid'][0] - 1
test_ids = split['tstid'][0] - 1

# 5- Cria Classes personalizadas do Dataset / Create Custom Dataset Class

Para carregar os dados no DataLoader do Pytorch, defina uma classe personalizada no DataSet

To load the data into PyTorch's DataLoader, define a custom dataset class.

In [6]:
class FlowersDataset(Dataset):
    def __init__(self, img_dir, indices, labels, transform=None):
        self.img_dir = img_dir
        self.indices = indices
        self.labels = labels
        self.transform = transform
        self.images = sorted(os.listdir(img_dir))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        image_id = self.indices[idx]
        img_path = os.path.join(self.img_dir, self.images[image_id])
        image = Image.open(img_path).convert('RGB')
        label = self.labels[image_id]
        if self.transform:
            image = self.transform(image)
        return image, label

# 6- Cria Transforms e DataLoaders / Create Transforms and DataLoaders

Pré-processa cada imagem para:

*  Redimensiona para 224x224
*  Normaliza usando a média e o padrão do ImageNet(ja que usei um modedlo pré-treinado).

Preprocess each image to:


*  Resize it to 224x224
*  Normalize it using ImageNet mean and std (since i used a pre-trained model).





In [7]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_ds = FlowersDataset(img_dir, train_ids, labels, transform)
val_ds = FlowersDataset(img_dir, val_ids, labels, transform)
test_ds = FlowersDataset(img_dir, test_ids, labels, transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

# 7- Carrega Resnet50 Pré-treinado / Load Pretained Resnet50

Eu usei um resnet 50 pré-treinado do modelzoo do PyTorch. Este modelo foi treinado no ImageNet e já reconhece padrões gerais em  imagens.

I used a pre-trained resnet50 from PyTorch's modelzoo. This model was trained on ImageNet and alredy knows how to recognize general patterns in images.

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = True

model.fc = nn.Linear(model.fc.in_features, 102) # Número de classes é 102
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 117MB/s]


# 8- Defina Perda e Otimizador / Define Loss and Optimizer

Eu usei Cross Entropy Loss e Adam Optimizer

I used Cross Entropy Loss and Adam optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 9- Função de Treinamento e Validação / Training and Validation Function

Treinei por alguns períodos, monitorando a perda de treinamento e a precisão da validação após cada um.

I trained for a few epochs, tracking training loss and validation accuracy after each.

In [10]:
def validate_model(model, loader):
    model.eval()
    preds, truths = [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            truths.extend(labels.numpy())
    acc = accuracy_score(truths, preds)
    print(f'Validation Accuracy: {acc*100:.2f}%')

def train_model(model, train_loader, val_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}')
        validate_model(model, val_loader)


# 10- Treinando o Modelo / Train the Model

In [11]:
train_model(model, train_loader, val_loader, epochs=10)

100%|██████████| 32/32 [12:47<00:00, 23.99s/it]


Epoch 1, Loss: 4.1804
Validation Accuracy: 57.35%


100%|██████████| 32/32 [12:07<00:00, 22.73s/it]


Epoch 2, Loss: 2.3418
Validation Accuracy: 79.41%


100%|██████████| 32/32 [12:06<00:00, 22.70s/it]


...

Epoch 8, Loss: 0.0384
Validation Accuracy: 91.18%


100%|██████████| 32/32 [11:52<00:00, 22.27s/it]


Epoch 9, Loss: 0.0268
Validation Accuracy: 91.86%


100%|██████████| 32/32 [11:51<00:00, 22.22s/it]


Epoch 10, Loss: 0.0233
Validation Accuracy: 92.35%


# 11- Avaliando no conjunto de teste / Evaluate on Test Set

In [12]:
def test_model(model, loader):
    model.eval()
    preds, truths = [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            truths.extend(labels.numpy())
    acc = accuracy_score(truths, preds)
    print(f'Test Accuracy: {acc*100:.2f}%')

test_model(model, test_loader)

KeyboardInterrupt: 